# 2buk

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423
┌ Info: Precompiling MDToolbox [7965e2f0-a35b-11e8-0207-19c030dd2def]
└ @ Base loading.jl:1423


In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1427]
        t2 = t[:, 1428:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

,fnames,score,omega,phi,prou
,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1030.45,"[-0.326659, 0.165874, -0.930473]",179.415,"[57.8244, 39.5924, -13.2422]"
2,1/complex.2.pdb,1015.76,"[0.393657, -0.0198818, 0.919042]",176.804,"[54.2806, 40.2127, -22.3803]"
3,1/complex.3.pdb,993.393,"[0.221836, 0.782683, -0.581546]",136.978,"[27.2611, 7.94716, 21.0948]"
4,1/complex.4.pdb,992.586,"[0.694194, 0.200992, 0.691156]",178.979,"[19.1993, 51.3184, -34.2073]"
5,1/complex.5.pdb,974.541,"[-0.365869, 0.842931, 0.394471]",136.337,"[59.9759, 32.9846, -14.8564]"
6,1/complex.6.pdb,973.094,"[-0.626489, -0.636017, -0.450548]",76.1917,"[13.37, 1.13562, -20.1942]"
7,1/complex.7.pdb,956.116,"[0.600767, 0.585152, -0.54468]",72.2297,"[-4.20772, 17.7384, 14.4154]"
8,1/complex.8.pdb,942.109,"[-0.326751, 0.941865, -0.0782578]",143.517,"[67.925, 23.1666, -4.78897]"
9,1/complex.9.pdb,939.108,"[0.651239, 0.499149, 0.571611]",114.402,"[21.6021, 9.96341, -33.3117]"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)